In [1]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from deep_macrofin import PDEModel, plot_loss_df
from deep_macrofin import ActivationType, Comparator, Constraint, System, set_seeds, OptimizerType, SamplingMethod

In [ ]:
set_seeds(0)
pde_model = PDEModel("BruSan", {"batch_size": 50, "num_epochs": 5000, "optimizer_type": OptimizerType.Adam, "sampling_method": SamplingMethod.FixedGrid})
pde_model.set_state(["e", "z"], {"e": [0.05, 0.95], "z": [0.05, 0.95]})
pde_model.add_endogs(["q", "psi", "mue", "sigqk", "sigqs"], configs={
    "q": {"positive": True},
    "psi": {"positive": True},
})
pde_model.add_agents(["vi", "vh"], configs={
    "vi": {"positive": True},
    "vh": {"positive": True},
})
pde_model.add_params({
    "gammai": 2,
    "gammah": 3,
    "ai": .1,
    "ah": .1,
    "rhoi": .04,
    "rhoh": .04,
    "sigz": .01,
    "sigbar": .5,
    "deltai": .04,
    "deltah": .04,
    "kappa_p": 2,
    "kappa_z": 5,
    "zetai": 1.15,
    "zetah": 1.15,
    "kappa_l": .9,
    "ebar": .5,
})

pde_model.add_equation("sigma = z")
pde_model.add_equation("wi = psi/e")
pde_model.add_equation("wh = (1-psi)/(1-e)")
pde_model.add_equation("ci = vi**((1-zetai)/(1-gammai))")
pde_model.add_equation("ch = vh**((1-zetah)/(1-gammah))")
pde_model.add_equation("iotai = (q-1)/kappa_p")
pde_model.add_equation("iotah = (q-1)/kappa_p")
pde_model.add_equation("phii = log(1+kappa_p*iotai)/kappa_p-deltai")
pde_model.add_equation("phih = log(1+kappa_p*iotah)/kappa_p-deltah")
pde_model.add_equation("muz = kappa_z*(sigbar-sigma)")
pde_model.add_equation("muk = psi*phii+(1-psi)*phih")
pde_model.add_equation("signis = wi*sigqs")
pde_model.add_equation("signhs = wh*sigqs")
pde_model.add_equation("signik = wi*(sigqk+sigma)")
pde_model.add_equation("signhk = wh*(sigqk+sigma)")
pde_model.add_equation("siges = e*(1-e)*(signis -sigqs)")
pde_model.add_equation("sigek = e*(1-e)*(signik - (sigqk+sigma))")
pde_model.add_equation("sigxik = vi_e/vi*sigek*e")
pde_model.add_equation("sigxhk = vh_e/vh*sigek*e")
pde_model.add_equation("sigxis = vi_e/vi*siges*e + vi_z/vi*sigz*z")
pde_model.add_equation("sigxhs = vh_e/vh*siges*e + vh_z/vh*sigz*z")
pde_model.add_equation("muq = q_e/q*mue*e + q_z/q*muz*z + 1/2*q_ee/q*((siges*e)**2 + (sigek*e)**2) + 1/2*q_zz/q*(sigz*z)**2 + q_ez/q*siges*e*sigz*z")
pde_model.add_equation("muri = (ai-iotai)/q + phii + muq + sigma*sigqk")
pde_model.add_equation("murh = (ah-iotah)/q + phih + muq + sigma*sigqk")
pde_model.add_equation("r = muri - gammai*wi*((sigqs**2)+(sigma+sigqk)**2) + sigqs*sigxis + (sigqk+sigma)*sigxik")
pde_model.add_equation("muni = r + wi*(muri-r)-ci")
pde_model.add_equation("munh = r + wh*(murh-r)-ch")
pde_model.add_equation("muxi = vi_e/vi*mue*e + vi_z/vi*muz*z + 1/2*vi_ee/vi*((siges*e)**2 + (sigek*e)**2) + 1/2*vi_zz/vi*(sigz*z)**2 + vi_ez/vi*siges*e*sigz*z")
pde_model.add_equation("muxh = vh_e/vh*mue*e + vh_z/vh*muz*z + 1/2*vh_ee/vh*((siges*e)**2 + (sigek*e)**2) + 1/2*vh_zz/vh*(sigz*z)**2 + vh_ez/vh*siges*e*sigz*z")

pde_model.add_endog_equation("kappa_l/e*(ebar-e)+(1-e)*(muni - muk - muq\
                     - sigma*sigqk + (sigqk+sigma)**2 + sigqs**2 \
                     - wi*sigqs**2 - wi*(sigqk+sigma)**2) - mue=0")
pde_model.add_endog_equation("(ci*e+ch*(1-e))*q - psi*(ai-iotai) - (1-psi)*(ah-iotah)=0")
pde_model.add_endog_equation("muri - murh + gammah*wh*((sigqs**2)+(sigqk+sigma)**2) - \
                     gammai*wi*((sigqs)**2+(sigqk+sigma)**2) + sigqs*sigxis + \
                     (sigqk+sigma)*sigxik - sigqs*sigxhs - (sigqk+sigma)*sigxhk=0")
pde_model.add_endog_equation("(sigz*z*q_z + siges*e*q_e)-sigqs*q=0")
pde_model.add_endog_equation("sigek*e*q_e - sigqk*q=0")

pde_model.add_hjb_equation("1/(1-1/zetai)*(ci-(rhoi+kappa_l)) + r - ci + gammai/2*(wi * sigqs)**2 + gammai/2*(wi*sigma+wi*sigqk)**2 + muxi / (1-gammai)")
pde_model.add_hjb_equation("1/(1-1/zetah)*(ch-(rhoh+kappa_l)) + r - ch + gammah/2*(wh * sigqs)**2 + gammah/2*(wh*sigma+wh*sigqk)**2 + muxh / (1-gammah)")

print(pde_model)
if not os.path.exists("./models/2D_fixed_grid/model.pt"):
    pde_model.train_model("./models/2D_fixed_grid", "model.pt", True)
    pde_model.load_model(torch.load("./models/2D_fixed_grid/model_best.pt"))
    pde_model.eval_model(True)
else:
    pde_model.load_model(torch.load("./models/2D_fixed_grid/model_best.pt"))
    pde_model.eval_model(True)

In [ ]:
plot_loss_df(fn="./models/2D_fixed_grid/model_loss.csv", loss_plot_fn="./models/2D_fixed_grid/model_loss.png")

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(24, 12), subplot_kw={"projection": "3d"})
# "q", "psi", "mue", "sigqk", "sigqs"
pde_model.endog_vars["q"].plot("q", {"e": [0, 1], "z": [0, 1]}, ax=ax[0][0])
pde_model.endog_vars["psi"].plot("psi", {"e": [0, 1], "z": [0, 1]}, ax=ax[0][1])
pde_model.endog_vars["mue"].plot("mue", {"e": [0, 1], "z": [0, 1]}, ax=ax[0][2])
pde_model.endog_vars["sigqk"].plot("sigqk", {"e": [0, 1], "z": [0, 1]}, ax=ax[0][3])
pde_model.endog_vars["sigqs"].plot("sigqs", {"e": [0, 1], "z": [0, 1]}, ax=ax[1][0])
pde_model.agents["vi"].plot("vi", {"e": [0, 1], "z": [0, 1]}, ax=ax[1][1])
pde_model.agents["vh"].plot("vh", {"e": [0, 1], "z": [0, 1]}, ax=ax[1][2])
plt.tight_layout()
plt.show()

In [ ]:
pde_model.plot_vars(["q", "psi", "mue", "vi", "vh",
                     "sigqk", "esigqk=e*sigqk", "sigqs", "esigqs=e*sigqs", "r"], ncols=5)